In [1]:
import os
import xml.dom.minidom as dom
import numpy as np
import pandas as pd

In [2]:
with open('../00_Download_and_Preprocessing/bam_files.txt', 'r') as f:
    bam_files = [os.path.basename(fn) for fn in f.read().strip().split('\n')]
htz1_rep1_rep2_bam_dict = {
    'SWR1 (WT)': [bam_files[0], bam_files[3]],
    'swc3 (LH∆)': [bam_files[1], bam_files[4]],
    'swc3∆': [bam_files[2], bam_files[5]]
}
swr1_bam_dict = {
    'SWR1 (WT)': [bam_files[6]],
    'swc3 (LH∆)': [bam_files[8]],
    'swc3∆': [bam_files[7]]
}
strain_color_dict = {
    'SWR1 (WT)': '#BFBFBF',
    'swc3 (LH∆)': '#D357FE',
    'swc3∆': '#FF4013'
}

In [3]:
def sliding_window(arr, window):
    width = np.shape(arr)[1]
    val = np.sum(arr[:, :window], axis=1) / window
    smoothed_arr = np.zeros((2, width - window + 1))
    smoothed_arr[:, 0] = val
    for i in range(1, width - window + 1):
        val += (arr[:, window + i - 1] - arr[:, i - 1]) / window
        smoothed_arr[:, i] = val
    return smoothed_arr

In [4]:
width = 1100
smoothing = 31
svgw = 300
svgh = 300
svgx = (40, 280)
svgy = (30, 265)
zero_idx = (width - smoothing + 1) // 2

def xscale(x):
    return (x + 500) / 1000 * (svgx[1] - svgx[0]) + svgx[0]

xvals = xscale(np.arange(-500, 500))

In [5]:
def create_axes(document, svg):
    xlabel = svg.appendChild(document.createElement('text'))
    xlabel.setAttribute('font-size', '12')
    xlabel.setAttribute('x', str((svgx[0] + svgx[1]) // 2))
    xlabel.setAttribute('y', str(svgh - 5))
    xlabel.setAttribute('text-anchor', 'middle')
    xlabel.appendChild(document.createTextNode('Distance from +1 Nucleosome'))
    xminlabel = svg.appendChild(document.createElement('text'))
    xminlabel.setAttribute('font-size', '12')
    xminlabel.setAttribute('x', str(svgx[0]))
    xminlabel.setAttribute('y', str(svgy[1] + 15))
    xminlabel.setAttribute('text-anchor', 'middle')
    xminlabel.appendChild(document.createTextNode('-500'))
    xmaxlabel = svg.appendChild(document.createElement('text'))
    xmaxlabel.setAttribute('font-size', '12')
    xmaxlabel.setAttribute('x', str(svgx[1]))
    xmaxlabel.setAttribute('y', str(svgy[1] + 15))
    xmaxlabel.setAttribute('text-anchor', 'middle')
    xmaxlabel.appendChild(document.createTextNode('500'))
    ylabel = svg.appendChild(document.createElement('text'))
    ylabel.setAttribute('font-size', '12')
    ylabel.setAttribute('x', str(svgx[0] - 12))
    ylabel.setAttribute('y', str((svgy[0] + svgy[1]) / 2))
    ylabel.setAttribute('text-anchor', 'middle')
    ylabel.setAttribute('transform', 'rotate(-90 {} {})'.format(svgx[0] - 12, (svgy[0] + svgy[1]) / 2))
    ylabel.appendChild(document.createTextNode('Occupancy (AU)'))
    yminlabel = svg.appendChild(document.createElement('text'))
    yminlabel.setAttribute('font-size', '12')
    yminlabel.setAttribute('x', str(svgx[0] - 10))
    yminlabel.setAttribute('y', str(svgy[1]))
    yminlabel.setAttribute('text-anchor', 'middle')
    yminlabel.appendChild(document.createTextNode('-1'))
    ymaxlabel = svg.appendChild(document.createElement('text'))
    ymaxlabel.setAttribute('font-size', '12')
    ymaxlabel.setAttribute('x', str(svgx[0] - 10))
    ymaxlabel.setAttribute('y', str(svgy[0]))
    ymaxlabel.setAttribute('text-anchor', 'middle')
    ymaxlabel.appendChild(document.createTextNode('1'))

    bottom_axis_group = svg.appendChild(document.createElement('g'))
    xaxis = bottom_axis_group.appendChild(document.createElement('line'))
    xaxis.setAttribute('x1', str(svgx[0]))
    xaxis.setAttribute('y1', str(svgy[1]))
    xaxis.setAttribute('x2', str(svgx[1]))
    xaxis.setAttribute('y2', str(svgy[1]))
    xaxis.setAttribute('stroke', '#000000')
    xaxis.setAttribute('stroke-width', '1')
    for x in [-400, -300, -200, -100, 0, 100, 200, 300, 400]:
        tick = bottom_axis_group.appendChild(document.createElement('line'))
        tick.setAttribute('x1', str(xscale(x)))
        tick.setAttribute('y1', str(svgy[1]))
        tick.setAttribute('x2', str(xscale(x)))
        tick.setAttribute('y2', str(svgy[1] - 6))
        tick.setAttribute('stroke', '#000000')
        tick.setAttribute('stroke-width', '1')
    top_axis_group = svg.appendChild(bottom_axis_group.cloneNode(True))
    top_axis_group.setAttribute('transform', 'rotate(180 {} {})'.format((svgx[0] + svgx[1]) // 2, (svgy[0] + svgy[1]) / 2))
    middle_axis_group = svg.appendChild(bottom_axis_group.cloneNode(True))
    middle_axis_group.setAttribute('transform', 'translate(0 {})'.format((svgy[0] - svgy[1]) / 2))
    for tick in middle_axis_group.childNodes[1:]:
        tick.setAttribute('y1', str(svgy[1] + 6))

    left_axis_group = svg.appendChild(document.createElement('g'))
    yaxis = left_axis_group.appendChild(document.createElement('line'))
    yaxis.setAttribute('x1', str(svgx[0]))
    yaxis.setAttribute('y1', str(svgy[0]))
    yaxis.setAttribute('x2', str(svgx[0]))
    yaxis.setAttribute('y2', str(svgy[1]))
    yaxis.setAttribute('stroke', '#000000')
    yaxis.setAttribute('stroke-width', '1')
    right_axis_group = svg.appendChild(left_axis_group.cloneNode(True))
    right_axis_group.setAttribute('transform', 'rotate(180 {} {})'.format((svgx[0] + svgx[1]) // 2, (svgy[0] + svgy[1]) / 2))
    return document

In [6]:
def write_svg(strain_bam_dict, bed, target, y_max, class_label):
    document = dom.Document()
    svg = document.appendChild(document.createElement('svg'))
    svg.setAttribute('xmlns', 'http://www.w3.org/2000/svg')
    svg.setAttribute('font-family', 'Helvetica')
    svg.setAttribute('viewBox', '0 0 {} {}'.format(svgw, svgh))
    svg.setAttribute('baseProfile', 'full')

    defs = svg.appendChild(document.createElement('defs'))

    def yscale(y):
        return (y + y_max) / (2 * y_max) * (svgy[1] - svgy[0]) + svgy[0]
    
    comp_dir = '../data/TagPileup/{}/Composites'.format(bed)
    comp_plot_dir = '../03_Composite_Results/{}'.format(bed)
    if not os.path.exists(comp_plot_dir):
        os.mkdir(comp_plot_dir)
    for i, strain in enumerate(strain_bam_dict):
        comp = np.zeros((2, 1001))
        for bam in strain_bam_dict[strain]:
            comp += sliding_window(pd.read_csv('{}/{}_{}.out'.format(comp_dir, bam[:-4], bed), sep='\t', index_col=0).values, smoothing)[:, zero_idx - 500:zero_idx + 501]
        comp = np.flip(comp, axis=0)
        strain_grad = defs.appendChild(document.createElement('linearGradient'))
        strain_grad.setAttribute('id', 'grad{}'.format(i))
        strain_grad.setAttribute('x1', '0%')
        strain_grad.setAttribute('y1', '0%')
        strain_grad.setAttribute('x2', '0%')
        strain_grad.setAttribute('y2', '100%')
        stop1 = strain_grad.appendChild(document.createElement('stop'))
        stop1.setAttribute('offset', '0%')
        stop1.setAttribute('stop-color', strain_color_dict[strain])
        stop1.setAttribute('stop-opacity', '1')
        stop2 = strain_grad.appendChild(document.createElement('stop'))
        stop2.setAttribute('offset', '{}%'.format(np.max(comp[1, :]) * 100 / (np.max(comp[0, :]) + np.max(comp[1, :]))))
        stop2.setAttribute('stop-color', strain_color_dict[strain])
        stop2.setAttribute('stop-opacity', '0')
        stop3 = strain_grad.appendChild(document.createElement('stop'))
        stop3.setAttribute('offset', '100%')
        stop3.setAttribute('stop-color', strain_color_dict[strain])
        stop3.setAttribute('stop-opacity', '1')
        comp_group = svg.appendChild(document.createElement('g'))
        comp_poly = comp_group.appendChild(document.createElement('polygon'))
        comp_points_sense = [(x, yscale(y)) for x, y in zip(xvals, comp[0, :])]
        comp_points_anti = [(x, yscale(-y)) for x, y in zip(np.flip(xvals), np.flip(comp[1, :]))]
        comp_poly.setAttribute('points', ' '.join('{},{}'.format(*p) for p in comp_points_sense + comp_points_anti))
        comp_poly.setAttribute('fill', 'url(#grad{})'.format(i))
        comp_poly.setAttribute('stroke', '#FFFFFF')
        comp_poly.setAttribute('stroke-width', '1')
        comp_outline_sense = comp_group.appendChild(document.createElement('path'))
        comp_outline_sense.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in comp_points_sense))
        comp_outline_sense.setAttribute('fill', 'none')
        comp_outline_sense.setAttribute('stroke', '#000000')
        comp_outline_sense.setAttribute('stroke-width', '0.5')
        comp_outline_anti = comp_group.appendChild(document.createElement('path'))
        comp_outline_anti.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in comp_points_anti))
        comp_outline_anti.setAttribute('fill', 'none')
        comp_outline_anti.setAttribute('stroke', '#000000')
        comp_outline_anti.setAttribute('stroke-width', '0.5')
        
        strain_label = svg.appendChild(document.createElement('text'))
        strain_label.setAttribute('font-size', '12')
        strain_label.setAttribute('text-anchor', 'left')
        strain_label.setAttribute('x', str(svgx[0] + 5))
        strain_label.setAttribute('y', str(svgy[0] + 42 + 12 * i))
        strain_label.setAttribute('fill', strain_color_dict[strain])
        strain_label.appendChild(document.createTextNode(strain))

    ref_label = svg.appendChild(document.createElement('text'))
    ref_label.setAttribute('font-size', '12')
    ref_label.setAttribute('text-anchor', 'left')
    ref_label.setAttribute('transform', 'translate({} {})'.format(svgx[0] + 5, svgy[0] + 18))
    class_row = ref_label.appendChild(document.createElement('tspan'))
    class_row.setAttribute('x', '0')
    class_row.setAttribute('y', '0')
    class_row.appendChild(document.createTextNode(class_label[0]))
    n_row = ref_label.appendChild(document.createElement('tspan'))
    n_row.setAttribute('x', '0')
    n_row.setAttribute('y', '12')
    n_row.appendChild(document.createTextNode('N = {}'.format(class_label[1])))

    zero_line = svg.appendChild(document.createElement('line'))
    zero_line.setAttribute('x1', str(xscale(0)))
    zero_line.setAttribute('x2', str(xscale(0)))
    zero_line.setAttribute('y1', str(svgy[0]))
    zero_line.setAttribute('y2', str(svgy[1]))
    zero_line.setAttribute('stroke', 'gray')
    zero_line.setAttribute('stroke-width', '1')
    zero_line.setAttribute('stroke-dasharray', '5,5')
    zero_line.setAttribute('opacity', '0.5')

    create_axes(document, svg)
    with open('{}/{}_{}.svg'.format(comp_plot_dir, target, class_label[0]), 'w') as f:
        document.writexml(f, addindent='    ', newl='\n')

In [7]:
write_svg(htz1_rep1_rep2_bam_dict, 'FEAT-Pol-II_RefPT+1Nuc___SubFEAT-TFIID-dom___4779_SORT-Sua7occ', 'Swr1', .1, ('Constitutive', 4779))
write_svg(swr1_bam_dict, 'FEAT-Pol-II_RefPT+1Nuc___SubFEAT-TFIID-dom___4779_SORT-Sua7occ', 'Swr1', .042, ('Constitutive', 4779))
write_svg(htz1_rep1_rep2_bam_dict, 'FEAT-Pol-II_RefPT+1Nuc__SubFEAT-YPD-25-37C-coding_01-RP___137_UNSORTED_1000bp', 'Htz1', .1, ('Ribosomal Protein', 137))
write_svg(htz1_rep1_rep2_bam_dict, 'FEAT-Pol-II_RefPT+1Nuc___SubFEAT-25C_M02a__150_SORT-Sua7occ_1000bp', 'Htz1', .1, ('Induced', 150))
write_svg(htz1_rep1_rep2_bam_dict, 'FEAT-Pol-II_RefPT+1Nuc___SubFEAT-25C_M03a__150_SORT-Sua7occ_1000bp', 'Htz1', .1, ('Poised', 150))
write_svg(htz1_rep1_rep2_bam_dict, 'FEAT-Pol-II_RefPT+1Nuc___SubFEAT-25C_M04__150_SORT-Sua7occ_1000bp', 'Htz1', .1, ('Constitutive', 150))